# 🎯 Advanced Grasp Detection v8

## New in v8
| Feature | Description |
|---------|-------------|
| **🔬 PCA-based Grasp** | ใช้ Principal Component Analysis หาแกนหลักของวัตถุ |
| **📐 True Width Measurement** | วัดความกว้างจริงตามแนวตั้งฉากกับแกนหลัก |
| **🦾 Tighter Grip** | grip_w - 5mm สำหรับการจับที่แน่นขึ้น |

## 1️⃣ Imports

In [ ]:
import sys
import cv2
import numpy as np
import time
import socket
import serial
import torch
from collections import deque
from ultralytics import YOLO

sys.path.append('Depth-Anything-V2')

print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
print("✓ Imports")

## 2️⃣ Hardware Configuration

In [ ]:
ROBOT_IP = '192.168.1.6'
ESP32_PORT = 'COM9'
ESP32_BAUDRATE = 115200
CAMERA_ID = 2

HOMOGRAPHY_MATRIX = np.array([
    [0.005703976266962427, -0.3265299161278153, 88.58634169557483],
    [-0.47704058225560797, 0.015355046930804153, 172.0941543570439],
    [-0.00029949919510557677, 0.00018728182448344945, 1.0],
], dtype=np.float32)

print("✓ Hardware config")

---
# 🔧 CALIBRATION SECTION
---

## 📐 Calibration 1: PIXELS_PER_MM

In [ ]:
# Uncomment เพื่อ calibrate
'''
drawing = False
start_pt, end_pt = None, None
px_dist = 0

def cb(event, x, y, flags, param):
    global drawing, start_pt, end_pt, px_dist
    if event == cv2.EVENT_LBUTTONDOWN:
        drawing, start_pt = True, (x, y)
    elif event == cv2.EVENT_MOUSEMOVE and drawing:
        end_pt = (x, y)
    elif event == cv2.EVENT_LBUTTONUP:
        drawing, end_pt = False, (x, y)
        px_dist = np.sqrt((end_pt[0]-start_pt[0])**2 + (end_pt[1]-start_pt[1])**2)
        print(f"Distance: {px_dist:.1f} px")

cap = cv2.VideoCapture(CAMERA_ID)
cv2.namedWindow('Pixel Cal')
cv2.setMouseCallback('Pixel Cal', cb)
while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break
    if start_pt and end_pt: cv2.line(frame, start_pt, end_pt, (0,255,0), 2)
    cv2.imshow('Pixel Cal', frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'): break
    elif key == 13 and px_dist > 0:
        mm = float(input("Real size (mm): "))
        print(f"PIXELS_PER_MM = {px_dist/mm:.4f}")
cap.release()
cv2.destroyAllWindows()
'''

## 📏 Calibration 2: Camera Height & Depth Scale

In [ ]:
CAMERA_HEIGHT_MM = 630  # วัดความสูงกล้องจากพื้น
print(f"Camera height: {CAMERA_HEIGHT_MM}mm")

---
## 3️⃣ Configuration
---

In [ ]:
# === CALIBRATED VALUES ===
PIXELS_PER_MM = 2.222
DEPTH_Z_SCALE = 26.1660
CAMERA_HEIGHT_MM = 630

# === Z Heights ===
Z_FLOOR = -64
Z_SAFE = -40
Z_APPROACH = -55

# === Drop Position ===
DROP_POS = (-253.07, 115.17, -17.07, -62.78)

# === Gripper ===
GRIPPER_SERVO_OPEN_ANGLE = 22
GRIPPER_SERVO_CLOSE_ANGLE = 96
GRIPPER_MAX_WIDTH_MM = 54
GRIPPER_MIN_WIDTH_MM = 0
GRIPPER_OPEN_MARGIN_MM = 5
GRIPPER_GRIP_MARGIN_MM = 5  # ใหม่: -5mm สำหรับจับแน่นขึ้น

# === Detection ===
MIN_OBJECT_AREA = 1000
YOLO_CONFIDENCE = 0.25

# === Depth Model ===
DEPTH_MODEL_PATH = 'Depth-Anything-V2/checkpoints/depth_anything_v2_vits.pth'
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# === Grid Display ===
SHOW_FRAME_GRID = True
SHOW_OBJECT_GRID = True
FRAME_GRID_SIZE_MM = 20
OBJECT_GRID_SIZE_MM = 5

print("✓ Configuration")
print(f"  GRIP_MARGIN: -{GRIPPER_GRIP_MARGIN_MM}mm (tighter grip)")

## 4️⃣ Load Models

In [ ]:
from depth_anything_v2.dpt import DepthAnythingV2

model_configs = {
    'vits': {'encoder': 'vits', 'features': 64, 'out_channels': [48, 96, 192, 384]},
}

print("Loading DepthAnything V2...")
depth_model = DepthAnythingV2(**model_configs['vits'])
depth_model.load_state_dict(torch.load(DEPTH_MODEL_PATH, map_location='cpu'))
depth_model = depth_model.to(DEVICE).eval()
print(f"✅ Depth model on {DEVICE}")

print("Loading YOLOv8...")
yolo_model = YOLO('yolov8n.pt')
print("✅ YOLO loaded")

## 📏 Multi-Object Depth Calibration (Optional)

In [ ]:
# Run นี้เมื่อต้องการ calibrate depth
'''
print("="*60)
print("📏 DEPTH CALIBRATION")
print("C=Floor | Click=Add Sample | S=Show | Q=Quit")
print("="*60)

calibration_samples = []
floor_depth = None
click_x, click_y = None, None

def depth_cal_cb(event, x, y, flags, param):
    global click_x, click_y
    if event == cv2.EVENT_LBUTTONDOWN:
        click_x, click_y = x, y

cap = cv2.VideoCapture(CAMERA_ID)
cv2.namedWindow('Depth Cal')
cv2.setMouseCallback('Depth Cal', depth_cal_cb)
current_depth_map = None
fc = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break
    fc += 1
    if fc % 5 == 0:
        current_depth_map = depth_model.infer_image(frame)
    
    floor_str = f"Floor: {floor_depth:.4f}" if floor_depth else "Floor: NOT SET"
    cv2.putText(frame, f"{floor_str} | Samples: {len(calibration_samples)}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)
    if calibration_samples:
        cv2.putText(frame, f"Scale: {np.mean([s['scale'] for s in calibration_samples]):.2f}", (10, 460), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)
    cv2.imshow('Depth Cal', frame)
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'): break
    elif key == ord('c') and current_depth_map is not None:
        h, w = current_depth_map.shape
        floor_depth = np.median(current_depth_map[h//3:2*h//3, w//3:2*w//3])
        print(f"✅ Floor: {floor_depth:.4f}")
    elif key == ord('s') and calibration_samples:
        print(f"\n✅ DEPTH_Z_SCALE = {np.mean([s['scale'] for s in calibration_samples]):.4f}")
    
    if click_x is not None and current_depth_map is not None and floor_depth is not None:
        x, y = click_x, click_y
        region = current_depth_map[max(0,y-10):y+10, max(0,x-10):x+10]
        raw_diff = np.median(region) - floor_depth
        if raw_diff > 0:
            try:
                h_mm = float(input(f"Height at ({x},{y}) in mm: "))
                calibration_samples.append({'height_mm': h_mm, 'scale': h_mm / raw_diff})
                print(f"   Sample {len(calibration_samples)}: Scale={h_mm/raw_diff:.2f}")
            except: pass
        click_x, click_y = None, None

cap.release()
cv2.destroyAllWindows()
if calibration_samples:
    print(f"\n📋 DEPTH_Z_SCALE = {np.mean([s['scale'] for s in calibration_samples]):.4f}")
'''

## 5️⃣ Classes Definition

In [ ]:
class SmartGripperController:
    CALIB_ANGLES = np.array([22, 30, 40, 50, 60, 70, 80, 90, 96])
    CALIB_WIDTHS = np.array([54.0, 52.0, 48.0, 40.0, 32.0, 23.0, 12.0, 3.0, 0.0])
    
    def __init__(self, port='COM9', baudrate=115200):
        self.port = port
        self.baudrate = baudrate
        self.serial = None
        self.target_width = None
        
    def connect(self):
        try:
            self.serial = serial.Serial(self.port, self.baudrate, timeout=2)
            time.sleep(2)
            print(f"✅ Gripper on {self.port}")
            return True
        except Exception as e:
            print(f"❌ {e}")
            return False
    
    def disconnect(self):
        if self.serial: self.serial.close()
    
    def send_command(self, cmd):
        if self.serial:
            self.serial.write((cmd + '\n').encode())
            time.sleep(0.3)
    
    def mm_to_angle(self, width_mm):
        width = max(0.0, min(54.0, width_mm))
        return int(round(np.interp(width, self.CALIB_WIDTHS[::-1], self.CALIB_ANGLES[::-1])))
    
    def open_for_object(self, width_mm):
        self.target_width = width_mm
        open_w = min(54.0, width_mm + GRIPPER_OPEN_MARGIN_MM)
        angle = self.mm_to_angle(open_w)
        print(f"🦾 Open: {open_w:.1f}mm ({angle}°)")
        self.send_command(f'G{angle}')
    
    def grip_object(self, width_mm):
        # ใช้ -5mm สำหรับจับแน่นขึ้น
        grip_w = max(0.0, width_mm - GRIPPER_GRIP_MARGIN_MM)
        angle = self.mm_to_angle(grip_w)
        print(f"🦾 Grip: {width_mm:.1f}mm - {GRIPPER_GRIP_MARGIN_MM}mm = {grip_w:.1f}mm ({angle}°)")
        self.send_command(f'G{angle}')
    
    def release(self):
        open_w = min(54.0, (self.target_width or 30) + 10)
        self.send_command(f'G{self.mm_to_angle(open_w)}')
        self.target_width = None


class DobotControllerTCP:
    def __init__(self, homography_matrix=None):
        self.dashboard_port = 29999
        self.sock = None
        self.H = homography_matrix
        
    def connect(self, ip):
        try:
            self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            self.sock.settimeout(5)
            self.sock.connect((ip, self.dashboard_port))
            self.send_command("ClearError()")
            time.sleep(0.5)
            self.send_command("EnableRobot()")
            time.sleep(4)
            self.send_command("User(1)")
            self.send_command("Tool(1)")
            self.send_command("SpeedFactor(50)")
            print("✅ Robot connected!")
            return True
        except Exception as e:
            print(f"Error: {e}")
            return False

    def send_command(self, cmd):
        if self.sock:
            self.sock.send((cmd + "\n").encode("utf-8"))
            return self.sock.recv(1024).decode("utf-8")

    def home(self):
        print("🤖 HOME...")
        self.send_command("MovJ(-253.07, 115.17, -17.07, -62.78)")
        time.sleep(4)

    def move_to(self, x, y, z, r=0):
        cmd = f"MovJ({x},{y},{z},{r})"
        print(f"   → {cmd}")
        return self.send_command(cmd)
    
    def move_to_and_wait(self, x, y, z, r=0, wait=3):
        self.move_to(x, y, z, r)
        time.sleep(wait)
    
    def joint_move(self, j1=0, j2=0, j3=0, j4=0):
        cmd = f"JointMovJ({j1},{j2},{j3},{j4})"
        print(f"   → {cmd}")
        return self.send_command(cmd)
    
    def joint_move_and_wait(self, j1=0, j2=0, j3=0, j4=0, wait=3):
        self.joint_move(j1, j2, j3, j4)
        time.sleep(wait)

    def pixel_to_robot(self, u, v):
        if self.H is None: return None, None
        pt = np.array([u, v, 1], dtype=np.float32)
        res = np.dot(self.H, pt)
        return res[0]/res[2], res[1]/res[2]

print("✓ Gripper & Robot classes")

In [ ]:
class PreciseSizeDetector:
    """YOLO + Contour + Grid"""
    
    def __init__(self, yolo_model, pixels_per_mm):
        self.yolo = yolo_model
        self.ppm = pixels_per_mm
    
    def detect(self, frame):
        objects = []
        results = self.yolo(frame, conf=YOLO_CONFIDENCE, verbose=False)
        
        for r in results:
            for box in r.boxes:
                x1,y1,x2,y2 = map(int, box.xyxy[0])
                conf = float(box.conf[0])
                
                roi = frame[y1:y2, x1:x2]
                if roi.size == 0: continue
                
                gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
                _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
                contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                
                if contours:
                    cnt = max(contours, key=cv2.contourArea)
                    cnt = cnt + np.array([x1, y1])
                    rect = cv2.minAreaRect(cnt)
                    cx, cy = int(rect[0][0]), int(rect[0][1])
                    
                    objects.append({
                        'bbox': (x1, y1, x2-x1, y2-y1),
                        'center': (cx, cy),
                        'rect': rect,
                        'rect_size': rect[1],
                        'rect_angle': rect[2],
                        'contour': cnt,
                        'conf': conf,
                        'area': cv2.contourArea(cnt)
                    })
        
        if not objects:
            objects = self.edge_detect(frame)
        
        return objects
    
    def edge_detect(self, frame):
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        edges = cv2.Canny(cv2.GaussianBlur(gray, (5,5), 0), 50, 150)
        edges = cv2.dilate(edges, np.ones((3,3), np.uint8), iterations=2)
        contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        objects = []
        for cnt in contours:
            area = cv2.contourArea(cnt)
            if area > MIN_OBJECT_AREA:
                hull = cv2.convexHull(cnt)
                rect = cv2.minAreaRect(hull)
                x,y,w,h = cv2.boundingRect(hull)
                objects.append({
                    'bbox': (x,y,w,h),
                    'center': (x+w//2, y+h//2),
                    'rect': rect,
                    'rect_size': rect[1],
                    'rect_angle': rect[2],
                    'contour': hull,
                    'area': area
                })
        return sorted(objects, key=lambda o: o['area'], reverse=True)
    
    def draw_frame_grid(self, frame):
        if not SHOW_FRAME_GRID: return frame
        h, w = frame.shape[:2]
        grid_px = int(FRAME_GRID_SIZE_MM * self.ppm)
        for x in range(0, w, grid_px):
            cv2.line(frame, (x,0), (x,h), (50,50,50), 1)
        for y in range(0, h, grid_px):
            cv2.line(frame, (0,y), (w,y), (50,50,50), 1)
        return frame
    
    def draw_object_grid(self, frame, obj):
        if not SHOW_OBJECT_GRID: return
        x, y, w, h = obj['bbox']
        grid_px = max(3, int(OBJECT_GRID_SIZE_MM * self.ppm))
        for gx in range(x, x+w, grid_px):
            cv2.line(frame, (gx, y), (gx, y+h), (100,100,255), 1)
        for gy in range(y, y+h, grid_px):
            cv2.line(frame, (x, gy), (x+w, gy), (100,100,255), 1)
        w_mm, h_mm = w / self.ppm, h / self.ppm
        cv2.putText(frame, f"{w_mm:.0f}x{h_mm:.0f}mm", (x, y-3), cv2.FONT_HERSHEY_SIMPLEX, 0.35, (100,100,255), 1)

print("✓ Detector class")

In [ ]:
class PCAGraspSelector:
    """v8: ใช้ PCA หาแกนหลักของวัตถุเพื่อ grasp ที่แม่นยำขึ้น"""
    
    def __init__(self, pixels_per_mm):
        self.ppm = pixels_per_mm
    
    def analyze_object(self, obj):
        """วิเคราะห์วัตถุด้วย PCA และ contour analysis"""
        cnt = obj.get('contour')
        if cnt is None or len(cnt) < 5:
            return self.fallback_analysis(obj)
        
        # PCA Analysis
        pts = cnt.reshape(-1, 2).astype(np.float64)
        mean = np.mean(pts, axis=0)
        pts_centered = pts - mean
        
        # Covariance และ eigenvectors
        cov = np.cov(pts_centered.T)
        eigenvalues, eigenvectors = np.linalg.eig(cov)
        
        # เรียงตาม eigenvalue (แกนหลักคือ eigenvector ที่มี eigenvalue สูงสุด)
        idx = np.argsort(eigenvalues)[::-1]
        eigenvalues = eigenvalues[idx]
        eigenvectors = eigenvectors[:, idx]
        
        # แกนหลัก (ทิศทางยาว) และแกนรอง (ทิศทางแคบ)
        major_axis = eigenvectors[:, 0]  # แกนยาว
        minor_axis = eigenvectors[:, 1]  # แกนแคบ
        
        # หามุมของแกนหลัก
        angle_major = np.degrees(np.arctan2(major_axis[1], major_axis[0]))
        
        # วัดความกว้างตามแนวแกนรอง (ตั้งฉากกับแกนยาว)
        # Project ทุกจุดลงบนแกนรอง
        projections = np.dot(pts_centered, minor_axis)
        width_px = np.max(projections) - np.min(projections)
        width_mm = width_px / self.ppm
        
        # Project ลงบนแกนหลักเพื่อหาความยาว
        proj_major = np.dot(pts_centered, major_axis)
        length_px = np.max(proj_major) - np.min(proj_major)
        length_mm = length_px / self.ppm
        
        cx, cy = int(mean[0]), int(mean[1])
        
        # มุม grasp = มุมของแกนรอง (จับตามแนวแคบ)
        grasp_angle = self.normalize_angle(angle_major + 90)
        
        grasps = []
        
        # Grasp หลัก: จับตามแนวแคบ
        if width_mm <= GRIPPER_MAX_WIDTH_MM:
            grasps.append({
                'center': (cx, cy),
                'width_mm': width_mm,
                'angle': grasp_angle,
                'score': 1.0,
                'type': 'PCA_narrow',
                'axis_info': {'major': major_axis, 'minor': minor_axis, 'width_px': width_px, 'length_px': length_px}
            })
        
        # Grasp ทางเลือก: จับตามแนวยาว (ถ้าเป็นไปได้)
        if length_mm <= GRIPPER_MAX_WIDTH_MM:
            grasps.append({
                'center': (cx, cy),
                'width_mm': length_mm,
                'angle': self.normalize_angle(angle_major),
                'score': 0.5,
                'type': 'PCA_long',
                'axis_info': None
            })
        
        # ถ้าไม่มี grasp เลย ใช้ fallback
        if not grasps:
            return self.fallback_analysis(obj)
        
        return grasps
    
    def fallback_analysis(self, obj):
        """Fallback ถ้า PCA ไม่ได้ผล"""
        rect = obj.get('rect')
        if rect is None:
            return []
        
        (cx, cy), (w, h), angle = rect
        cx, cy = int(cx), int(cy)
        
        # จับด้านแคบสุด
        if w < h:
            grip_w = w / self.ppm
            grip_a = angle + 90
        else:
            grip_w = h / self.ppm
            grip_a = angle
        
        grasps = []
        if grip_w <= GRIPPER_MAX_WIDTH_MM:
            grasps.append({
                'center': (cx, cy),
                'width_mm': grip_w,
                'angle': self.normalize_angle(grip_a),
                'score': 0.8,
                'type': 'rect_fallback',
                'axis_info': None
            })
        
        return grasps
    
    def normalize_angle(self, angle):
        while angle > 90: angle -= 180
        while angle < -90: angle += 180
        return angle

print("✓ PCA Grasp Selector (v8)")

In [ ]:
class RobustDepthEstimator:
    def __init__(self, model, device='cpu', camera_height_mm=630, history_size=5):
        self.model = model
        self.device = device
        self.camera_height = camera_height_mm
        self.floor_depth = None
        self.history = deque(maxlen=history_size)
    
    def estimate_depth(self, frame):
        return self.model.infer_image(frame)
    
    def calibrate_floor(self, frame):
        depth = self.estimate_depth(frame)
        h, w = depth.shape
        self.floor_depth = np.median(depth[h//3:2*h//3, w//3:2*w//3])
        print(f"✅ Floor depth: {self.floor_depth:.4f}")
        return self.floor_depth
    
    def get_object_height(self, depth_map, obj, scale):
        if self.floor_depth is None: return 0
        x, y, w, h = obj['bbox']
        region = depth_map[y:y+h, x:x+w]
        if region.size == 0: return 0
        
        samples = [np.median(region)]
        qh, qw = h//4, w//4
        if qh > 0 and qw > 0:
            samples.extend([np.median(region[:qh,:qw]), np.median(region[:qh,-qw:]),
                           np.median(region[-qh:,:qw]), np.median(region[-qh:,-qw:])])
        
        height = max(0, (np.median(samples) - self.floor_depth) * scale)
        self.history.append(height)
        return np.median(self.history)
    
    def calculate_z(self, height_mm):
        return max(Z_FLOOR, min(Z_SAFE, Z_FLOOR + height_mm * 0.5))

print("✓ Depth estimator")

## 6️⃣ Initialize & Connect

In [ ]:
gripper = SmartGripperController(port=ESP32_PORT, baudrate=ESP32_BAUDRATE)
robot = DobotControllerTCP(homography_matrix=HOMOGRAPHY_MATRIX)
detector = PreciseSizeDetector(yolo_model, PIXELS_PER_MM)
grasp_selector = PCAGraspSelector(PIXELS_PER_MM)  # v8: ใช้ PCA
depth_estimator = RobustDepthEstimator(depth_model, device=DEVICE, camera_height_mm=CAMERA_HEIGHT_MM)
print("✓ Components initialized (v8 with PCA)")

In [ ]:
gripper.connect()

In [ ]:
robot.connect(ROBOT_IP)

## 📷 Capture Background

In [ ]:
print("📷 BACKGROUND | SPACE=Capture Q=Skip")
cap = cv2.VideoCapture(CAMERA_ID)
while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break
    cv2.putText(frame, "SPACE=Capture | Q=Skip", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,255), 2)
    cv2.imshow('BG', frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord(' '):
        depth_estimator.calibrate_floor(frame)
        break
    elif key == ord('q'):
        print("Skipped")
        break
cap.release()
cv2.destroyAllWindows()

---
# 🎯 MAIN PICK-AND-PLACE v8
---

In [ ]:
selected_object = None
selected_grasp = None
detected_objects = []
current_grasps = []
current_depth = None

def mouse_callback(event, x, y, flags, param):
    global selected_object, selected_grasp, current_grasps
    if event == cv2.EVENT_LBUTTONDOWN:
        # Check if clicking on grasp point
        for g in current_grasps:
            gx, gy = g['center']
            if abs(x-gx) < 20 and abs(y-gy) < 20:
                selected_grasp = g
                print(f"\n🎯 Grasp: W={g['width_mm']:.1f}mm A={g['angle']:.1f}° ({g['type']})")
                return
        # Otherwise select object
        for obj in detected_objects:
            bx,by,bw,bh = obj['bbox']
            if bx <= x <= bx+bw and by <= y <= by+bh:
                selected_object = obj
                current_grasps = grasp_selector.analyze_object(obj)
                selected_grasp = current_grasps[0] if current_grasps else None
                if selected_grasp:
                    print(f"\n📦 Object: {len(current_grasps)} grasps")
                    print(f"   Best: W={selected_grasp['width_mm']:.1f}mm A={selected_grasp['angle']:.1f}° ({selected_grasp['type']})")
                break

def draw_grasps(frame, obj, grasps, selected):
    for g in grasps:
        cx, cy = g['center']
        angle = g['angle']
        is_sel = (selected and g == selected)
        color = (0,0,255) if is_sel else ((0,255,0) if g['score']>=1.0 else (0,255,255))
        thick = 3 if is_sel else 2
        
        # Draw grasp line
        length = 40
        dx = int(length * np.cos(np.radians(angle)))
        dy = int(length * np.sin(np.radians(angle)))
        cv2.line(frame, (cx-dx, cy-dy), (cx+dx, cy+dy), color, thick)
        cv2.circle(frame, (cx, cy), 5, color, -1)
        
        # Draw gripper jaws
        grip_half = int(g['width_mm'] * PIXELS_PER_MM / 2)
        perp_angle = angle + 90
        px = int(grip_half * np.cos(np.radians(perp_angle)))
        py = int(grip_half * np.sin(np.radians(perp_angle)))
        
        # Jaw 1
        j1_start = (cx + px - dx//2, cy + py - dy//2)
        j1_end = (cx + px + dx//2, cy + py + dy//2)
        cv2.line(frame, j1_start, j1_end, color, 2)
        
        # Jaw 2
        j2_start = (cx - px - dx//2, cy - py - dy//2)
        j2_end = (cx - px + dx//2, cy - py + dy//2)
        cv2.line(frame, j2_start, j2_end, color, 2)
        
        # Width label
        if is_sel:
            cv2.putText(frame, f"{g['width_mm']:.1f}mm", (cx+10, cy-10),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.4, color, 1)

def pick_with_grasp(obj, grasp):
    cx, cy = grasp['center']
    grip_w = grasp['width_mm']
    robot_r = -grasp['angle']
    robot_x, robot_y = robot.pixel_to_robot(cx, cy)
    height = depth_estimator.get_object_height(current_depth, obj, DEPTH_Z_SCALE) if current_depth is not None else 0
    z_grasp = depth_estimator.calculate_z(height)
    
    print(f"\n🤖 Pick: W={grip_w:.1f}mm R={robot_r:.1f}° Z={z_grasp:.1f}")
    print(f"   Type: {grasp['type']}")
    
    print("🔄 Safe position...")
    robot.joint_move_and_wait(0, 0, 0, 0, 3)
    
    # v8: เปิด gripper เต็มที่เสมอ
    gripper.open_for_object(GRIPPER_MAX_WIDTH_MM)
    time.sleep(4)
    
    robot.move_to_and_wait(robot_x, robot_y, Z_APPROACH, robot_r, 3)
    robot.move_to_and_wait(robot_x, robot_y, z_grasp, robot_r, 2)
    
    # v8: grip_w - 5mm สำหรับจับแน่นขึ้น
    gripper.grip_object(grip_w - 5)
    time.sleep(4)
    
    robot.move_to_and_wait(robot_x, robot_y, Z_SAFE, robot_r, 2)
    robot.move_to_and_wait(*DROP_POS[:3], DROP_POS[3], 3)
    
    gripper.release()
    time.sleep(4)
    robot.home()
    print("✅ Complete!")

# Main loop
cap = cv2.VideoCapture(CAMERA_ID)
cv2.namedWindow('Pick v8 (PCA)')
cv2.setMouseCallback('Pick v8 (PCA)', mouse_callback)

frame_count = 0
print("="*60)
print("🎯 PICK v8 (PCA-based Grasp)")
print("Click=Select | SPACE=Execute | F=Grid | O=ObjGrid | Q=Quit")
print("="*60)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break
    
    frame = detector.draw_frame_grid(frame)
    
    frame_count += 1
    if frame_count % 10 == 0:
        current_depth = depth_estimator.estimate_depth(frame)
    
    detected_objects = detector.detect(frame)
    
    for obj in detected_objects:
        x,y,w,h = obj['bbox']
        is_sel = (selected_object and obj['center'] == selected_object['center'])
        color = (0,0,255) if is_sel else (0,255,0)
        
        cv2.rectangle(frame, (x,y), (x+w,y+h), color, 2)
        if 'rect' in obj:
            box = cv2.boxPoints(obj['rect'])
            cv2.drawContours(frame, [np.int32(box)], 0, color, 1)
        
        detector.draw_object_grid(frame, obj)
    
    if selected_object and current_grasps:
        draw_grasps(frame, selected_object, current_grasps, selected_grasp)
    
    # Status bar
    cv2.rectangle(frame, (0,0), (640,35), (30,30,30), -1)
    cv2.putText(frame, f"v8 PCA | Obj:{len(detected_objects)} | Click | SPACE | F/O=Grid | Q",
               (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255,255,255), 1)
    
    if selected_grasp:
        cv2.putText(frame, f"[{selected_grasp['type']}: W={selected_grasp['width_mm']:.1f}mm - SPACE]",
                   (10, 470), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 2)
    
    cv2.imshow('Pick v8 (PCA)', frame)
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'): break
    elif key == ord('r'):
        selected_object = None
        selected_grasp = None
        current_grasps = []
    elif key == ord('f'):
        SHOW_FRAME_GRID = not SHOW_FRAME_GRID
        print(f"Frame grid: {SHOW_FRAME_GRID}")
    elif key == ord('o'):
        SHOW_OBJECT_GRID = not SHOW_OBJECT_GRID
        print(f"Object grid: {SHOW_OBJECT_GRID}")
    elif key == ord('h'):
        robot.home()
    elif key == ord(' ') and selected_object and selected_grasp:
        pick_with_grasp(selected_object, selected_grasp)
        selected_object = None
        selected_grasp = None
        current_grasps = []

cap.release()
cv2.destroyAllWindows()

In [ ]:
robot.home()

In [ ]:
gripper.disconnect()
print("✅ Done")